In [1]:
import QuantLib as ql
import pandas as pd

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Parameters

In [39]:
calendar = ql.Argentina()
dayCount = ql.Thirty360()
compounding = ql.Compounded

### TO21

In [93]:
compoundingFrequency = ql.Semiannual
tenor = ql.Period(compoundingFrequency)
faceValue = 100
settlementDays = 0

# Intervalo de pagos del bond
start_date = ql.Date(3, 4, 2019)
end_date = ql.Date(3, 10, 2021)

schedule_prueba = ql.Schedule(start_date, end_date, tenor, calendar, ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, False)
print('SCHEDULE PRUEBA:', list(schedule_prueba))

# Cupones: ANUAL (hay que especificarlos anual, para que recalcule el 'amount' de cada cupon, segun el compoundingFrequency) ( VER PARAMETROS EXTRAS ... )
coupons = [0.182 for _ in schedule_prueba]

b = ql.FixedRateBond(settlementDays, calendar, faceValue, start_date, end_date, tenor, coupons, dayCount, ql.Unadjusted, ql.Unadjusted)
# b = ql.FixedRateBond(settlementDays, calendar, faceValue, start_date, end_date, tenor, coupons, dayCount, ql.Unadjusted)
# b = ql.FixedRateBond(settlementDays, calendar, faceValue, start_date, end_date, tenor, coupons, dayCount)

ql.Settings.instance().evaluationDate = ql.Date(3,4,2019)

# TODO: pricear usando spots de bonos gob
# print ('NPV(1):', b.NPV())
accrued_interest = b.accruedAmount()
bond_price = 72

print('ACCRUED:', accrued_interest)

ytm = b.bondYield(bond_price, dayCount, compounding, compoundingFrequency)

print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])
print ('YTM (anual):', ytm)

ytm = round(ytm, 7)
ytm_rate = ql.InterestRate(ytm, dayCount, ql.Compounded, compoundingFrequency)
print ('DURATION:', ql.BondFunctions.duration(b, ytm_rate, ql.Duration.Modified))

SCHEDULE PRUEBA: [Date(3,4,2019), Date(3,10,2019), Date(3,4,2020), Date(3,10,2020), Date(3,4,2021), Date(3,10,2021)]
ACCRUED: 0.0
PAYMENTS: [Date(3,10,2019), Date(3,4,2020), Date(3,10,2020), Date(3,4,2021), Date(3,10,2021), Date(3,10,2021)]
COUPONS: [9.1, 9.1, 9.1, 9.1, 9.1, 100.0]
YTM (anual): 0.3613493280147314
DURATION: 1.7283260787418753


In [79]:
# ql.FixedRateBond(settlementDays, calendar, faceValue, start_date, end_date, tenor, coupons)

### Fixed Rates (?)

In [86]:
# EJEMPLO: Fixed Rate: cupones iguales, schedule automatico. Usa spotCurveHandle
# dayCount ya definida
compoundingFrequency = ql.Semiannual
tenor = ql.Period(compoundingFrequency)
faceValue = 100
settlementDays = 0
couponRate = 0.06 # anual

ql.Settings.instance().evaluationDate = ql.Date(15,1,2015)
coupons = [couponRate]

# VER: ql.Unadjusted / ql.DateGeneration.Backword/Forwarde
schedule = ql.Schedule(ql.Date(15, 1, 2015), ql.Date(15, 7, 2017), tenor, calendar, 
                       ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Backward, True)

b = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount, ql.Unadjusted)

actual_price = 100
ytm = b.bondYield(actual_price, dayCount, compounding, compoundingFrequency)
ytm = round(ytm, 7)
print ('SCHEDULE:', list(schedule))
print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])
print ('YTM:', ytm)

SCHEDULE: [Date(15,1,2015), Date(15,7,2015), Date(15,1,2016), Date(15,7,2016), Date(15,1,2017), Date(15,7,2017)]
PAYMENTS: [Date(15,7,2015), Date(15,1,2016), Date(15,7,2016), Date(15,1,2017), Date(15,7,2017), Date(15,7,2017)]
COUPONS: [3.0, 3.0, 3.0, 3.0, 3.0, 100.0]
YTM: 0.06


In [87]:
# Fixed Rate: cupones diferentes, schedule automatico (mismo que ejemplo anterior). Usa spotCurveHandle
# YA DEFINIDOS: calendar, dayCount, compounding

compoundingFrequency = ql.Semiannual
tenor = ql.Period(compoundingFrequency)
faceValue = 100
settlementDays = 0

coupons = [.06,.06,.06, .06, .06] # ANUAL (hay que especificarlos anual, para que recalcule el 'amount' de cada cupon, segun el compoundingFrequency)
b = ql.FixedRateBond(settlementDays, calendar, faceValue, ql.Date(15, 1, 2015), ql.Date(15, 7, 2017), tenor, coupons, dayCount, ql.Unadjusted, ql.Unadjusted) # VER PARAMETROS EXTRAS ...

ql.Settings.instance().evaluationDate = ql.Date(15,1,2015)

# DURATION: Hay que pasarle el YTM (previamente calcularlo a partir de la curva spot)
ytm = b.bondYield(faceValue, dayCount, ql.Compounded, compoundingFrequency)
ytm = round(ytm, 7)
ytm_rate = ql.InterestRate(ytm, dayCount, ql.Compounded, compoundingFrequency)

print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])
print ('YTM (anual):', ytm)
print ('DURATION:', ql.BondFunctions.duration(b, ytm_rate, ql.Duration.Modified))

PAYMENTS: [Date(15,7,2015), Date(15,1,2016), Date(15,7,2016), Date(15,1,2017), Date(15,7,2017), Date(15,7,2017)]
COUPONS: [3.0, 3.0, 3.0, 3.0, 3.0, 100.0]
YTM (anual): 0.06
DURATION: 2.2898535935972664


### Fixed schedule interval + possible Variable coupon. Use Spot Curve

In [90]:
spotDates = [ql.Date(15, 1, 2015), ql.Date(15, 7, 2015), ql.Date(15, 1, 2016), ql.Date(15,7,2016), ql.Date(15,1,2017), ql.Date(15,7,2017)]

coupons   = [0.06, 0.06, 0.06, 0.06, 0.06, 0.06]
spotRates = [0.06, 0.06, 0.06, 0.06, 0.06, 0.06]


# Build Spot Curve
interpolation = ql.Linear()
spotCurve = ql.ZeroCurve(spotDates, spotRates, dayCount, calendar, interpolation, compounding)
spotCurveHandle = ql.YieldTermStructureHandle(spotCurve)
bondEngine = ql.DiscountingBondEngine(spotCurveHandle)
compoundingFrequency = ql.Semiannual
tenor = ql.Period(compoundingFrequency)

b = ql.FixedRateBond(settlementDays, calendar, faceValue, ql.Date(15, 1, 2015), ql.Date(15, 7, 2017), tenor, coupons, dayCount, ql.Unadjusted, ql.Unadjusted) # VER PARAMETROS EXTRAS ...
b.setPricingEngine(bondEngine)

ql.Settings.instance().evaluationDate = ql.Date(15,1,2015)

print('ACCRUED:', b.accruedAmount())
      
print('SPOT DATES:', spotDates)
print ('PAYMENTS:', [cf.date() for cf in b.cashflows()])
print ('COUPONS:', [round(cf.amount(),2) for cf in b.cashflows()])

print ('NPV(1):', b.NPV())

tot = faceValue*coupons[1]/2/pow(1+spotRates[1], 0.5)
tot += faceValue*coupons[2]/2/pow(1+spotRates[2], 1)
tot += faceValue*coupons[3]/2/pow(1+spotRates[3], 1.5)
tot += faceValue*coupons[4]/2/pow(1+spotRates[4], 2)
tot += (faceValue + faceValue*coupons[5]/2)/pow(1+spotRates[5], 2.5)
print ('NPV(2):', tot)

ytm = b.bondYield(100, dayCount, ql.Compounded, compoundingFrequency)
ytm = round(ytm, 2)
print ('YTM:', ytm)

ACCRUED: 0.0
SPOT DATES: [Date(15,1,2015), Date(15,7,2015), Date(15,1,2016), Date(15,7,2016), Date(15,1,2017), Date(15,7,2017)]
PAYMENTS: [Date(15,7,2015), Date(15,1,2016), Date(15,7,2016), Date(15,1,2017), Date(15,7,2017), Date(15,7,2017)]
COUPONS: [3.0, 3.0, 3.0, 3.0, 3.0, 100.0]
NPV(1): 100.2003766909293
NPV(2): 100.20037669092929
YTM: 0.06


In [9]:
# b.cashflows()
# help(ql.BondFunctions)
# ejs.: ql.BondFunctions.zSpread, ...

### AA22: ver otra forma. Tiene schedule con intervalo no fijo

In [23]:
b.__dir__()

['settlementValue',
 'setPricingEngine',
 'settlementDate',
 '__reduce__',
 '__setattr__',
 '__dir__',
 'redemption',
 '__ne__',
 '__doc__',
 'calendar',
 'previousCouponRate',
 'this',
 'thisown',
 '__deref__',
 '__reduce_ex__',
 'recalculate',
 'notionals',
 '__bool__',
 '__lt__',
 '__repr__',
 '__init__',
 '__sizeof__',
 'accruedAmount',
 '__str__',
 '__getattribute__',
 'notional',
 'errorEstimate',
 '__eq__',
 'nextCouponRate',
 'issueDate',
 '__delattr__',
 'unfreeze',
 '__new__',
 'isExpired',
 '__format__',
 'cashflows',
 'redemptions',
 '__swig_destroy__',
 '__weakref__',
 '__subclasshook__',
 'settlementDays',
 'cleanPrice',
 '__le__',
 'asObservable',
 'maturityDate',
 '__gt__',
 'dirtyPrice',
 '__hash__',
 '__class__',
 'bondYield',
 '__nonzero__',
 '__del__',
 'NPV',
 'startDate',
 '__ge__',
 'freeze',
 'dayCounter',
 '__dict__',
 '__module__',
 'frequency']